<a href="https://colab.research.google.com/github/DataDrivenAkanksha/Life-expectancy-deep-learning/blob/main/life_expactancy_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment ANN CNN -Akanksha Rawat

a) What are vanishing and exploding gradients? How do they affect neural networks? Explain in detail.

Vanishing Gradient Problem
The vanishing gradient problem occurs during backpropagation when the gradients (partial derivatives of the loss with respect to weights) become very small—especially in deep neural networks.

Cause: This typically happens when using activation functions like sigmoid or tanh, where the derivative is less than 1. When many such small derivatives are multiplied (as in the chain rule), the resulting gradient becomes extremely small.

Effect: Weights in earlier layers are updated very slowly (or not at all), effectively stalling learning in those layers.

Consequence: The network fails to learn deep features, making it hard to train deep networks effectively.

Example: If each derivative is ~0.1 and you have 100 layers, the gradient becomes ~0.1¹⁰⁰ → effectively zero.

Exploding Gradient Problem
The exploding gradient problem is the opposite: gradients become very large during backpropagation.

Cause: This is often due to large weight values or activation functions like ReLU (though ReLU itself isn't the direct cause—it's more about the weights and initialization). When derivatives > 1 are multiplied across many layers, the gradient grows exponentially.

Effect: Weight updates become huge, causing the network to become unstable and possibly diverge (loss goes to infinity or NaN).

Consequence: The model cannot converge, and training fails.

Example: If each derivative is ~2 and you have 100 layers, the gradient becomes 2¹⁰⁰ → extremely large.

Solutions
For vanishing gradients:

Use ReLU, Leaky ReLU, or ELU instead of sigmoid/tanh.

Use Batch Normalization.

Use residual connections (like in ResNet).

Use proper weight initialization (e.g., Xavier or He initialization).

For exploding gradients:

Apply gradient clipping.

Use Batch Normalization.

Monitor and adjust learning rates.

Use proper weight initialization.

b) Use the Life Expectancy Prediction dataset from below Kaggle link and create an end-to-end project on Jupyter/Colab to predict the life expectancy.

https://www.kaggle.com/code/ranasabrii/life-expectancy-regression-with-ann/notebook


i. Download the dataset from above link and load it into your Python environment.

ii. Perform the EDA and do the visualizations.

iii. Check the distributions/skewness in the variables and do the transformations if required.

iv. Check/Treat the outliers and do the feature scaling if required.

v. Build Deep Learning model using ANN with multiple hidden layers.

vi. Apply the dropout regularization and early stopping techniques to improve model performance.

vii. Use the modelCheckpoint also to store the parameters after each epoch.

viii. Use the KerasTuner to tune to best parameters (No. of hidden layers, optimizers, loss function, activation functions etc.)

ix. Compare the accuracies of different models and finalize the best model.

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [8]:
!kaggle datasets download -d kumarajarshi/life-expectancy-who

Dataset URL: https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who
License(s): other
  0% 0.00/119k [00:00<?, ?B/s]
100% 119k/119k [00:00<00:00, 308MB/s]


In [9]:
import zipfile
zip_ref = zipfile.ZipFile('/content/life-expectancy-who.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [11]:
import pandas as pd
import numpy as np
df=pd.read_csv('/content/Life Expectancy Data.csv')
df.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5
